In [21]:
import pandas as pd
import os
from math import *
import re
from datetime import datetime, date, time, timedelta
import fix_yahoo_finance as yf
from time import sleep

#universe = ['spy','ark'] 
universe = ['spy'] 
#variables for data download
frame = 20 #for limiting the range of optimizations, 1 year
hist_window = frame * 5 #for historical pricing
date_fmt = '%Y-%m-%d'
start_date = datetime.now() - timedelta(hist_window)
start_date = start_date.strftime(date_fmt)
sleep_time = 5

## Script que obtiene los datos de las fuentes:
# 1. Sector components   : Contiene la información de los ETF sectors y sus componentes
# 2. Pricing            : Contiene la informacion historica de precios de cada uno de los componentes

In [22]:
################################################################################################################################
### 1. Sector components:
# de donde se extrae esta info, con que frecuencia se actualiza?

component_path = "./sector_components/"

## 1.1 descargar componentes del server:
#### aqui de donde se descargan los datos???????

## 1.2 cargar los sector componentes para cada benchmark, cargar cada ETF sector para cada benchmark
# y cada company para cada ETF sectior y limpiarlos

def load_companies(benchmark,skiprows,columns): ## el input es el nombre del benchmark, si ignora o no el primer row y el nombre de las columnas que son de interes, primero ticker y luego nombre de la compañia
    resultados_benchmark = {}
    resultados_companies = {}
    resultados = pd.DataFrame([])
    ## que ETF SECTOR tiene el benchmark?: calcularlo:
    directory=component_path+benchmark ## directorio donde se buscará los archivos
    file_list=os.listdir(directory)    ## lista de archivos en el directorio
    files = [f for f in file_list if bool(re.search("-"+benchmark+"-", f))] ## lista de archivos .etf sectors
    for key in files: ## loop que recorre todos los etf sectors
        aux = pd.read_csv(directory+"/"+key,skiprows=skiprows)[columns] ## load companies
        aux.columns=["ticker","company_name"]
        aux["benchmark"]=benchmark ## agrega columna con nombre del benchmark
        aux["etf_sector"]=re.search("\-([a-z]*?)\.csv",key).groups()[0]  # agrega columna con nonmbre del etf sector
        print("loaded: "+ re.search("\-([a-z]*?)\.csv",key).groups()[0])
        resultados=resultados.append(aux)
        resultados_companies[re.search("\-([a-z]*?)\.csv",key).groups()[0]] = aux.ticker.tolist()
    resultados_benchmark[benchmark]=resultados.etf_sector.unique().tolist()
    return resultados,resultados_benchmark,resultados_companies


companies_spy,benchmark_spy_etf_map,spy_sector_tickers_map=load_companies(benchmark="spy",skiprows=1,columns=['Symbol', 'Company Name']) ## companies del benchmark spy
print("Companies loaded:", companies_spy.shape)

#companies_ark,benchmark_ark_etf_map1,ark_sector_tickers_map1=load_companies(benchmark="ark",skiprows=0,columns=['ticker','company']) ## companies del benchmark ark
#print("Companies loaded:", companies1.shape)

loaded: xlb
loaded: xle
loaded: xlf
loaded: xli
loaded: xlk
loaded: xlp
loaded: xlre
loaded: xlu
loaded: xlv
loaded: xly
Companies loaded: (505, 4)


In [23]:
### aqui generalizar para mas benchmarks
companies_universe=companies_spy.copy()
sector_etf = benchmark_spy_etf_map.copy()
ticker_map=spy_sector_tickers_map.copy()

In [24]:
################################################################################################################################
# 2. Princing            : Contiene la informacion historica de precios de cada uno de los componentes

## 2.1 descargar componentes del server:

pricing_path = "./pricing/"

def get_pricing(output_name,ticker_list,start_date):
    try:
        if isinstance(ticker_list, list)==False:
            ticker_list=[ticker_list]
        px = yf.download(ticker_list, start=start_date)['Adj Close']
        if isinstance(px, pd.DataFrame)==False: 
            px=px.to_frame()
            px.columns=ticker_list
        px.sort_index(ascending=True, inplace=True)
        px.to_csv(output_name)
        print(output_name)
        #return px
    except Exception as err:
        print("Error: {0}, waiting to try again in {1}".format(err, sleep_time))
        print(output_name)
        sleep(sleep_time)




In [25]:
### refresh components
for i in universe:
    for j in sector_etf[i]:
        [i,j,ticker_map[j]]
etfs=[[i,j,ticker_map[j]] for j in sector_etf[i] for i in universe]
for i in etfs:
    get_pricing(output_name=pricing_path+i[0]+"/"+i[1]+"-hold-pricing.csv",
        ticker_list=i[2],start_date=start_date)

[*********************100%***********************]  25 of 25 downloaded
./pricing/spy/xlb-hold-pricing.csv
[*********************100%***********************]  32 of 32 downloaded
./pricing/spy/xle-hold-pricing.csv
[*********************100%***********************]  67 of 67 downloaded
./pricing/spy/xlf-hold-pricing.csv
[*********************100%***********************]  70 of 70 downloaded
./pricing/spy/xli-hold-pricing.csv
[*********************100%***********************]  71 of 71 downloaded
./pricing/spy/xlk-hold-pricing.csv
[*********************100%***********************]  34 of 34 downloaded
./pricing/spy/xlp-hold-pricing.csv
[*********************100%***********************]  33 of 33 downloaded
./pricing/spy/xlre-hold-pricing.csv
[*********************100%***********************]  28 of 28 downloaded
./pricing/spy/xlu-hold-pricing.csv
[*********************100%***********************]  61 of 61 downloaded
./pricing/spy/xlv-hold-pricing.csv
[*********************100%**********

In [26]:
## refresh etf sectors 
sectors=[[i,sector_etf[i]] for i in universe]
for j in sectors:
    get_pricing(output_name=pricing_path+"/"+j[0]+"/"+j[0]+'-sectors.csv', 
        ticker_list=j[1], start_date=start_date)

[*********************100%***********************]  10 of 10 downloaded
./pricing//spy/spy-sectors.csv


In [27]:
# refresh spy index
get_pricing(output_name=pricing_path+'/spy/SPY.csv', ticker_list='SPY', start_date=start_date)


[*********************100%***********************]  1 of 1 downloaded
./pricing//spy/SPY.csv
